In [1]:

import pandas as pd 
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,  GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import gc
from tune_sklearn import TuneSearchCV


# Обучение модели с CatBoost на данных с методом трансформации классов

In [2]:
df= pd.read_csv(r'X5_clients_data_train.csv', sep = ';')
med = df.age.median()
df.age = df.age.apply(lambda x: x if((x >= 16) and (x <= 90)) else med)
df.age = df.age.fillna(med)
df.Mean_diffs = df.Mean_diffs.fillna(df.Mean_diffs.median()) 
df = df.fillna(0)
df.describe()
display(df)

In [ ]:
df = df.drop(['MeanItemCost', 'client_id'
                , 'Mean_TOP_1_BRAND'
                ,	'Mean_TOP_2_BRAND'
                ,	'Mean_TOP_3_BRAND'
                ,	'Mean_TOP_4_BRAND',
                	'Mean_TOP_5_BRAND',
                    'Mean_TOP_1_SEG',
                    'Mean_TOP_2_SEG',
                    'Mean_TOP_3_SEG',
                    'Mean_TOP_4_SEG',
                    'Mean_TOP_5_SEG',
                    'first_order_date'	], axis = 1)
# X = df.iloc[:, 0:-1].set_index('client_id')
# y = df.iloc[:, [0, -1]].set_index('client_id')
# del df
# gc.collect()
treat = df.treatment_flg.values
targ = df.target.values
treat

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [ ]:

p = df[df.treatment_flg == 1].shape[0] / df.shape[0]
X = df.iloc[:, 0:-2].values
X = (X - X.mean()) / X.std()
y = df.target * (df.treatment_flg - p) / (1- p) / p
y = y.values
del df
gc.collect()
print(X)
print(y)
print(p)
# X_train, X_test, y_train, y_test = train_test_split(X, y)
# output.enable_custom_widget_manager()
# grid = {'learning_rate':  np.linspace(0, 0.05, 10)[1:-1],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9]}
# clf = CatBoostClassifier(verbose = 0)  
# grid_search_result = clf.grid_search(grid, 
#                                        X=X_train, 
#                                        y=y_train, 
#                                        plot=True)
from sklift.metrics import (
    uplift_at_k, uplift_auc_score, qini_auc_score, weighted_average_uplift
)

from sklift.viz import plot_qini_curve, plot_uplift_curve, plot_uplift_by_percentile
import matplotlib.pyplot as plt


[[-0.14500764 -0.15234497 -0.15234497 ... -0.14118879 -0.15234497
  -0.15277405]
 [-0.14324839 -0.15234497 -0.14977047 ... -0.13732704 -0.15234497
  -0.15277405]
 [-0.14110297 -0.15234497 -0.15234497 ... -0.14247604 -0.1514868
  -0.15234497]
 ...
 [-0.1438062  -0.15234497 -0.13689795 ... -0.13389437 -0.15234497
  -0.15277405]
 [-0.14715306 -0.15234497 -0.15234497 ... -0.13990154 -0.15234497
  -0.15234497]
 [-0.12085022 -0.15234497 -0.15234497 ... -0.14161787 -0.15234497
  -0.15277405]]
[-1.99923045  0.          0.         ... -1.99923045  0.
 -1.99923045]
0.4998075375301816


In [ ]:

X_train,X_test,y_train,y_test= train_test_split(X, y)

In [ ]:


param_grid = {
    'max_depth': [2, 4, 6, 10, 20],
    
    'min_data_in_leaf': [1, 2, 3, 4, 6],
    'min_child_samples': [2, 4, 8],
    'n_estimators': [10, 50, 100]
}# Create a based model
rf = CatBoostRegressor(verbose = 1)# Instantiate the grid search model
grid_search = TuneSearchCV(estimator = rf, param_distributions= param_grid, 
                           n_jobs = 4, verbose = 2)

In [ ]:
grid_search.fit(X_train, y_train)

2022-12-26 15:48:52,306	INFO worker.py:1538 -- Started a local Ray instance.


KeyboardInterrupt: 

In [ ]:
best = grid_search.best_estimator_

In [ ]:
grid_search.best_params_

In [ ]:
# # Метод двух моделей
# #Default data
# skf = KFold(n_splits=5)
# skf.get_n_splits(X, y)
# metrics = {'wau':[], 'uplift_k':[]}
# i = 0

# for train_index, test_index in skf.split(X, y):
#       gc.collect()
#       X_train_, X_test_ = X[train_index], X[test_index]
#       y_train_, y_test_ = y[train_index], y[test_index]
#       reg = CatBoostRegressor(verbose=0)
#       reg.fit(X_train_, y_train_)
#       y_pred = reg.predict(X_test_)
#       del reg
#       del X_train_
#       del y_train_
#       del X_test_
#       gc.collect()
#       wau = weighted_average_uplift(y_true=targ[test_index] , uplift=y_pred,
#                               treatment=treat[test_index])
#       uplift_overall = uplift_at_k(y_true=targ[test_index] , uplift=y_pred,
#                               treatment=treat[test_index], strategy='by_group', k=0.3)
#       metrics['wau'].append(wau)
#       metrics['uplift_k'].append(uplift_overall)
#       i += 1
#       print(f'Split {i}; WAU = {wau}; UpLift 30% = {uplift_overall}')
#       fig, ax = plt.subplots(1, 2, figsize = (30, 10))
#       ax[0].set_title('Qini Curve')
#       ax[1].set_title('UpLift Curve')
#       plot_qini_curve(y_true=targ[test_index] , uplift=y_pred,
#                               treatment=treat[test_index], ax = ax[0], perfect=False)
#       gc.collect()
#       plot_uplift_curve(y_true=targ[test_index] , uplift=y_pred,
#                               treatment=treat[test_index], ax = ax[1], perfect=False)
#       gc.collect()
#       gc.collect()
#       gc.collect()
#       plt.savefig(f"Plots/Regression model Split {i}.png")

#       plot_uplift_by_percentile(y_true=targ[test_index], uplift=y_pred,
#                               treatment=treat[test_index], kind='bar')
#       plt.savefig(f"Plots/Regression model Procentile Split {i}.png")


# wau = np.round(np.mean(metrics['wau']), 4)
# uplift_overall = np.round(np.mean(metrics['uplift_k']), 4)
# print(f'Mean WAU = {wau}; Mean UpLift 30% = {uplift_overall}')


In [ ]:
gc.collect()

71372